In [1]:
import pandas as pd
import numpy as np
import wmfdata as wmf

from datetime import timedelta
import warnings

## spark_session

In [2]:
spark_session = wmf.spark.get_active_session()

if type(spark_session) != type(None):
    spark_session.stop()
else:
    print('no active session')

no active session


In [ ]:
spark_session = wmf.spark.create_session()

In [5]:
spark_session.sparkContext.setLogLevel("ERROR")

## analysis

In [31]:
query = """
WITH
    sessions AS (
        SELECT
            event.editing_session_id AS session_id,
            event.platform,
            event.page_ns,
            event.user_class,
            SUM(CAST(event.action = 'init' AS INT)) AS init_count,
            SUM(CAST(event.action = 'saveSuccess' AS INT)) AS save_success_count,
            SUM(CAST(event.action = 'abort' AS INT)) AS abort_count,
            SUM(CAST(event.action = 'saveFailure' AS INT)) AS save_failure_count,
            MAX(dt) AS max_ts,
            MIN(dt) AS min_ts,
            month
        FROM
            event_sanitized.editattemptstep
        WHERE
            event.editor_interface = 'visualeditor' AND
            NOT event.is_oversample AND
            year = 2023 AND month <=6
        GROUP BY
            event.editing_session_id,
            event.platform,
            event.page_ns,
            event.user_class,
            month
    )

SELECT
    month,
    COUNT(DISTINCT session_id) AS all_sessions,
    COUNT(DISTINCT (CASE WHEN init_count = 0 THEN session_id END)) AS zero_inits,
    COUNT(DISTINCT (CASE WHEN init_count > 1 THEN session_id END)) AS multi_inits,
    COUNT(DISTINCT (CASE WHEN (init_count >0 AND save_success_count = 0 AND abort_count = 0 AND save_failure_count = 0) THEN session_id END)) AS zero_outcomes,
    COUNT(DISTINCT (CASE WHEN (save_success_count > 1 AND abort_count > 1) THEN session_id END)) AS save_abort
FROM 
    sessions 
GROUP BY 
    month
ORDER BY
    month
"""

result = wmf.spark.run(query)

In [32]:
result

,month,all_sessions,zero_inits,multi_inits,zero_outcomes,save_abort
0,1,10539487,224724,3538,1552832,189
1,2,9221142,195755,3155,1329948,197
2,3,10037346,212781,3533,1459611,210
3,4,9449683,200310,4949,2328152,201
4,5,9951207,203356,5078,2476765,223
5,6,8762817,189110,4237,2168110,146
